In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Define Functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

def split_series(series, input_size, output_size, train_ratio, seed):
    # 1. split univariate series to input (X) and output (y)
    X, y = [], []  
    for i in range(len(series) - input_size - output_size + 1):
        X.append(series[i:i + input_size])                            # X = input e.g. [10, 20, 30]
        y.append(series[i + input_size:i + input_size + output_size]) # y = output e.g. [40, 50]
    # 2. shuffle batches and split into train/test
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size = train_ratio, random_state=seed)
    
    return X_train, X_test, Y_train, Y_test

# Quantile loss function
def quantile_loss(preds, targets, quantiles):
    losses = []
    for i, quantile in enumerate(quantiles):
        errors = targets[:, :, i] - preds[:, :, i]
        losses.append(torch.mean(torch.max((quantile - 1) * errors, quantile * errors)))
    return torch.mean(torch.stack(losses))

def evaluate_model(model, dataloader, quantiles):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    
    with torch.no_grad():  # Disable gradient calculation
        for inputs, targets in dataloader:
            inputs = inputs.unsqueeze(-1)  # Adjust dimensions if necessary
            targets = targets.unsqueeze(-1).expand(-1, -1, len(quantiles))  # Reshape targets
            outputs = model(inputs)  # Forward pass
            all_preds.append(outputs)
            all_targets.append(targets)
    
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    return all_preds, all_targets

class BDLSTM_Quantile(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_quantiles, num_steps_ahead):
        super(BDLSTM_Quantile, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_quantiles = num_quantiles
        self.num_steps_ahead = num_steps_ahead

        self.lstms = nn.ModuleList()
        self.lstms.append(nn.LSTM(input_size, hidden_size[0], batch_first=True, bidirectional=True))
        for i in range(1, num_layers):
            self.lstms.append(nn.LSTM(hidden_size[i-1] * 2, hidden_size[i], batch_first=True, bidirectional=True))
        
        # Define separate linear layers for each quantile
        self.fc = nn.ModuleList([nn.Linear(hidden_size[-1] * 2, self.num_steps_ahead) for _ in range(num_quantiles)])
        self.dropout = nn.Dropout(0.2)  # Dropout layer with 20% dropout rate

    def forward(self, x):
        for lstm in self.lstms:
            x, _ = lstm(x)
        
        lstm_out = x[:, -1, :]  # Use the output of the last time step
        lstm_out = self.dropout(lstm_out)  # Apply dropout
        
        # Compute the outputs for each quantile
        quantile_outputs = [fc(lstm_out) for fc in self.fc]
        
        # Stack the quantile outputs
        output = torch.stack(quantile_outputs, dim=2)
        return output

pm = "\u00B1"

# Load and preprocess the data
Bitcoin = pd.read_csv('data/coin_Ethereum.csv')
Close_Price = Bitcoin['Close'].copy()
Close_Price_reshaped = np.array(Close_Price).reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
Close_Price_scaled = scaler.fit_transform(Close_Price_reshaped).flatten()

input_size = 1
hidden_size = [50, 50]
num_layers = 2
output_size = 5
train_ratio = 0.8
seed = 5925
num_experiments = 30
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
num_quantiles = len(quantiles)

rmse, mae, mape = [], [], []
train_rmse, train_mae, train_mape = [], [], []
rmse_005, rmse_025, rmse_050, rmse_075, rmse_095 = [], [], [], [], []
train_rmse_005, train_rmse_025, train_rmse_050, train_rmse_075, train_rmse_095 = [], [], [], [], []
rmse_steps = [[] for _ in range(output_size)]
train_rmse_steps = [[] for _ in range(output_size)]
mae_steps = [[] for _ in range(output_size)]
train_mae_steps = [[] for _ in range(output_size)]
mape_steps = [[] for _ in range(output_size)]
train_mape_steps = [[] for _ in range(output_size)]

for exp in range(num_experiments):
    X_train, X_test, y_train, y_test = split_series(Close_Price_scaled, input_size, output_size, train_ratio, seed)
    train_dataset = Time_Series_Dataset(X_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    model = BDLSTM_Quantile(input_size, hidden_size, num_layers, num_quantiles, output_size)

    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_dataloader:
            inputs = inputs.unsqueeze(-1).float()
            targets = targets.unsqueeze(-1).expand(-1, output_size, num_quantiles).float()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = quantile_loss(outputs, targets, quantiles)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
        
        if epoch % 10 == 0 or epoch == num_epochs:
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.6f}')

    # Evaluate the model on the training set
    train_predicted_values, train_actual_values = evaluate_model(model, train_dataloader, quantiles)
    train_predicted_values = train_predicted_values.numpy()
    train_actuals = train_actual_values.numpy()[:, :, 0]

    train_pred_005 = train_predicted_values[:, :, 0]
    train_pred_025 = train_predicted_values[:, :, 1]
    train_pred_050 = train_predicted_values[:, :, 2]
    train_pred_075 = train_predicted_values[:, :, 3]
    train_pred_095 = train_predicted_values[:, :, 4]

    train_mse_005 = mean_squared_error(train_pred_005.flatten(), train_actuals.flatten())
    train_mse_025 = mean_squared_error(train_pred_025.flatten(), train_actuals.flatten())
    train_mse_050 = mean_squared_error(train_pred_050.flatten(), train_actuals.flatten())
    train_mse_075 = mean_squared_error(train_pred_075.flatten(), train_actuals.flatten())
    train_mse_095 = mean_squared_error(train_pred_095.flatten(), train_actuals.flatten())

    train_rmse_005.append(np.sqrt(train_mse_005))
    train_rmse_025.append(np.sqrt(train_mse_025))
    train_rmse_050.append(np.sqrt(train_mse_050))
    train_rmse_075.append(np.sqrt(train_mse_075))
    train_rmse_095.append(np.sqrt(train_mse_095))

    train_pred_values = train_predicted_values.reshape(-1, 1)
    train_pred_values = scaler.inverse_transform(train_pred_values)
    train_pred_values = train_pred_values.reshape(train_predicted_values.shape)
    train_actual_values = scaler.inverse_transform(train_actuals.reshape(-1, 1)).reshape(train_actuals.shape)

    train_predicted_005 = train_pred_values[:, :, 0]
    train_predicted_025 = train_pred_values[:, :, 1]
    train_predicted_050 = train_pred_values[:, :, 2]
    train_predicted_075 = train_pred_values[:, :, 3]
    train_predicted_095 = train_pred_values[:, :, 4]

    train_mae.append(mean_absolute_error(train_actual_values.flatten(), train_predicted_050.flatten()))
    train_mape.append(mean_absolute_percentage_error(train_actual_values.flatten(), train_predicted_050.flatten()))

    for step in range(output_size):
        train_mse_step = mean_squared_error(train_pred_050[:, step], train_actuals[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))
        train_mae_steps[step].append(mean_absolute_error(train_predicted_050[:, step], train_actual_values[:, step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_predicted_050[:, step], train_actual_values[:, step]))

    # Evaluate the model on the test set
    predicted_values, actual_values = evaluate_model(model, test_dataloader, quantiles)
    predicted_values = predicted_values.numpy()
    actuals = actual_values.numpy()[:, :, 0]

    pred_005 = predicted_values[:, :, 0]
    pred_025 = predicted_values[:, :, 1]
    pred_050 = predicted_values[:, :, 2]
    pred_075 = predicted_values[:, :, 3]
    pred_095 = predicted_values[:, :, 4]

    mse_005 = mean_squared_error(pred_005.flatten(), actuals.flatten())
    mse_025 = mean_squared_error(pred_025.flatten(), actuals.flatten())
    mse_050 = mean_squared_error(pred_050.flatten(), actuals.flatten())
    mse_075 = mean_squared_error(pred_075.flatten(), actuals.flatten())
    mse_095 = mean_squared_error(pred_095.flatten(), actuals.flatten())

    rmse_005.append(np.sqrt(mse_005))
    rmse_025.append(np.sqrt(mse_025))
    rmse_050.append(np.sqrt(mse_050))
    rmse_075.append(np.sqrt(mse_075))
    rmse_095.append(np.sqrt(mse_095))

    pred_values = predicted_values.reshape(-1, 1)
    pred_values = scaler.inverse_transform(pred_values)
    pred_values = pred_values.reshape(predicted_values.shape)
    actual_values = scaler.inverse_transform(actuals.reshape(-1, 1)).reshape(actuals.shape)

    predicted_005 = pred_values[:, :, 0]
    predicted_025 = pred_values[:, :, 1]
    predicted_050 = pred_values[:, :, 2]
    predicted_075 = pred_values[:, :, 3]
    predicted_095 = pred_values[:, :, 4]

    mae.append(mean_absolute_error(actual_values.flatten(), predicted_050.flatten()))
    mape.append(mean_absolute_percentage_error(actual_values.flatten(), predicted_050.flatten()))

    for step in range(output_size):
        mse_step = mean_squared_error(pred_050[:, step], actuals[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))
        mae_steps[step].append(mean_absolute_error(predicted_050[:, step], actual_values[:, step]))
        mape_steps[step].append(mean_absolute_percentage_error(predicted_050[:, step], actual_values[:, step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

print(f'BD-LSTM Training Average RMSE across {output_size} time steps at different quantiles:')
print(f'Quantile 0.05: {np.mean(train_rmse_005):.6f} {pm} {np.std(train_rmse_005):.6f}')
print(f'Quantile 0.25: {np.mean(train_rmse_025):.6f} {pm} {np.std(train_rmse_025):.6f}')
print(f'Quantile 0.50: {np.mean(train_rmse_050):.6f} {pm} {np.std(train_rmse_050):.6f}')
print(f'Quantile 0.75: {np.mean(train_rmse_075):.6f} {pm} {np.std(train_rmse_075):.6f}')
print(f'Quantile 0.95: {np.mean(train_rmse_095):.6f} {pm} {np.std(train_rmse_095):.6f}')

print("--------------------------------------------")

print(f"Univariate Quantile Regression Training Average Performance: RMSE: {np.mean(train_rmse_050):.4f} {pm} {np.std(train_rmse_050):.4f}, MAE: {np.mean(train_mae):.4f} {pm} {np.std(train_mae):.4f}, MAPE: {np.mean(train_mape)*100:.4f}% {pm} {np.std(train_mape)*100:.4f}%")
for step in range(output_size):
    print(f"At time step {step + 1}, training predictions have RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, MAE: {np.mean(train_mae_steps[step]):.4f} {pm} {np.std(train_mae_steps[step]):.4f} and MAPE: {np.mean(train_mape_steps[step])*100:.4f}% {pm} {np.std(train_mape_steps[step])*100:.4f}%")

print(f'BD-LSTM Test Average RMSE across {output_size} time steps at different quantiles:')
print(f'Quantile 0.05: {np.mean(rmse_005):.6f} {pm} {np.std(rmse_005):.6f}')
print(f'Quantile 0.25: {np.mean(rmse_025):.6f} {pm} {np.std(rmse_025):.6f}')
print(f'Quantile 0.50: {np.mean(rmse_050):.6f} {pm} {np.std(rmse_050):.6f}')
print(f'Quantile 0.75: {np.mean(rmse_075):.6f} {pm} {np.std(rmse_075):.6f}')
print(f'Quantile 0.95: {np.mean(rmse_095):.6f} {pm} {np.std(rmse_095):.6f}')

print("--------------------------------------------")

print(f"Univariate Quantile Regression Test Average Performance: RMSE: {np.mean(rmse_050):.4f} {pm} {np.std(rmse_050):.4f}, MAE: {np.mean(mae):.4f} {pm} {np.std(mae):.4f}, MAPE: {np.mean(mape)*100:.4f}% {pm} {np.std(mape)*100:.4f}%")
for step in range(output_size):
    print(f"At time step {step + 1}, test predictions have RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, MAE: {np.mean(mae_steps[step]):.4f} {pm} {np.std(mae_steps[step]):.4f} and MAPE: {np.mean(mape_steps[step])*100:.4f}% {pm} {np.std(mape_steps[step])*100:.4f}%")


Epoch [10/100], Loss: 0.024166
Epoch [20/100], Loss: 0.004583
Epoch [30/100], Loss: 0.003755
Epoch [40/100], Loss: 0.003284
Epoch [50/100], Loss: 0.003068
Epoch [60/100], Loss: 0.003056
Epoch [70/100], Loss: 0.002963
Epoch [80/100], Loss: 0.002805
Epoch [90/100], Loss: 0.002822
Epoch [100/100], Loss: 0.002791
Experiment 1/30 done
Epoch [10/100], Loss: 0.024301
Epoch [20/100], Loss: 0.004741
Epoch [30/100], Loss: 0.003793
Epoch [40/100], Loss: 0.003266
Epoch [50/100], Loss: 0.003073
Epoch [60/100], Loss: 0.002936
Epoch [70/100], Loss: 0.002870
Epoch [80/100], Loss: 0.002768
Epoch [90/100], Loss: 0.002771
Epoch [100/100], Loss: 0.002812
Experiment 2/30 done
Epoch [10/100], Loss: 0.023449
Epoch [20/100], Loss: 0.004754
Epoch [30/100], Loss: 0.003540
Epoch [40/100], Loss: 0.003163
Epoch [50/100], Loss: 0.003012
Epoch [60/100], Loss: 0.002926
Epoch [70/100], Loss: 0.002819
Epoch [80/100], Loss: 0.002831
Epoch [90/100], Loss: 0.002794
Epoch [100/100], Loss: 0.002752
Experiment 3/30 done
Epoc

In [2]:
quantile_dfs = []
for i in range(pred_values.shape[2]):
    quantile_df = pd.DataFrame(
        pred_values[:, :, i], 
        columns=[f'Quantile_{i+1}_Timestep_{j+1}' for j in range(pred_values.shape[1])]
    )
    quantile_dfs.append(quantile_df)
predicted_df = pd.concat(quantile_dfs, axis=1)

predicted_df.to_csv('ethereum_uni_bdlstm_quantile_results.csv')
pd.DataFrame(actual_values).to_csv('ethereum_uni_bdlstm_quantile_actual.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('ethereum_uni_bdlstm_quantile_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('ethereum_uni_bdlstm_quantile_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('ethereum_uni_bdlstm_quantile_mape.csv')